### SDC Profile

### What features are driving student success in SDC application?

In [1]:
import psycopg2
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
question = pd.read_csv('admissions questions-nd013.csv')

### Exclude cohort 7
### Cohorts considered here: 47, 88, 89, 18  -- closed on Feb 21st

### Hypothesis Tests on Topics:
* Gender
* Degree relevant
* Education
* Education institution
* Professional experience
* Employer
* Employment
* Python
* C++
* Programming skills
* Theory skills
* Time commitment
* GitHub / linkedin account
* Goal
* Purpose
* Phone
* Location

### Result:

1. What works:
    * Education : phd > master > bachelor
    * Professional Experience: longer years of experience --> more tend to pay; but not necessarily completion rate; fpr completion rate, 5+ years > less than 1 year > 1-2 years > 3-5 years
    * Python: 
         * more experience --> more tend to pay; but not necessarily completion rate
         * for completion rate, students who have intermediate python experience have highest completion rate (1-2 years then 2+ years given measurement of years; level 3 then level 4 given measurement of levels)
         * we have 2600-ish students were asked using year measurement, 380-ish students were asked using level measurement. no significant difference detected yet. one thing worth mentioning is level measurement separates level 1 and higher better, and year measurement can only separate with and without experience.
    * C++: 
         * works well to predict payment (more experience --> more tend to pay)
         * not a good indicator for completion (students has no experience has a pretty good completion rate)
         * in terms of measurement of years or levels, same as python is level measurement has a better separation of completion rate than years
    * Purpose: grow skills for my current role > start a new career in this field / help move from academia to industry > general interest in the topic (personal growth...)
2. What not works:
    * Degree Relevant (question_id: cad2565e-8176-44b5-9080-14cfc67ea9b4)
    * Employment (fba3666b-db04-46e9-8f3d-2a303f13e0a5)
    * Programming Skills (e.g. depth-first search and breadth-first search, balanced trees. etc..)
    * Theory Skills (e.g. machine learning, calculus, linear algebra, probability and statistics, etc...)

### Findings:

1. Gender:
    * 1/3 of both male (35%) and female (33%) paid after being accepted
    * term completion rate of male is a little bit higher than female (50% vs. 44%)
2. Degree Relevant
    * no specific degrees impact payment after accepted
    * no specific degrees impact term completion
3. Education
    * PhD has 50% paid, followed by Master 39% (Bachelor 31%)
    * PhD has 57% term completion, followed by Master 51% (Bachelor 44%)
4. Professional Experience
    * 5+ years has 42% paid, followed by 3-5 years 37% (1-2+ years 33%)
    * 5+ years has 50% term completion, followed by < 1 year 49%, then 1-2+ years 49%, then 3-5 years 48%
5. Employment
    * not significant
6. Python
    * for payment, it seems the higher python experience is, the higher rate to pay
    * for term completion, separate by groups:
        * group 1 : use year as measurement (bigger group with 2600-ish students paid)
             * 1-2 years (51%) > 2+ years (51%) > < 1 year (50%) > no experience (36%)
        * group 2 : use level as measurement (smaller group with 380-ish students paid)
             * level 0 (2 out of 3) > level 3 (44%) > level 4 (43%) > level 2 (43%) > level 1 (20%)
7. C++
    * for payment, same as python, the higher experience, the higher rate to pay
    * for term completion, at lower groups are level 0,1,2; even no prior experience has a very good completion rate
8. Programming Skills (e.g. depth-first search and breadth-first search, balanced trees. etc..)
    * payment, as along as student selected at least one, paid rate is similar
    * term completion, no winner, as along as student selected at least one, completion rate is similar
9. Theory Skill (e.g. machine learning, calculus, linear algebra, probability and statistics, etc...)
    * payment: as long as one is selected, pay rate is not that different. highest is machine learning (38%) and lowest is computer science (36%)
    * term completion: 17 students selected "none of the above" and 9 of them completed term;  same as payment, machine learning (53%), computer science (49%)
10. Purpose
    * grow skills for my current role > start a new career in this field / help move from academia to industry > general interest in the topic (personal growth...)

In [3]:
sql_1 = """
SELECT distinct a.id,a.applicant_id,a.cohort_id,a.cohort_open_at,a.cohort_close_at,a.cohort_start_at
      ,a.application_created_at,a.submitted_at,a.accepted_at,a.first_charge_created_at
      ,'{tag}' AS tag,coalesce(lower(q.response),'no response') AS response
      ,CASE WHEN grad.graduated_at IS NOT NULL THEN 'graduated'
            WHEN b.user_id IS NOT NULL THEN 'term completed'
            WHEN a.first_charge_created_at IS NOT NULL THEN 'paid'
       ELSE 'accepted'
       END AS status
      ,CASE WHEN a.first_charge_created_at IS NOT NULL THEN 'paid' ELSE 'accepted' END AS paid_status
FROM applications a
LEFT JOIN admissions.question_responses q 
  ON a.id = q.application_id AND q.question_id IN ({question})
  AND a.cohort_id = q.cohort_id
LEFT JOIN registrar_v2.graduations grad ON a.applicant_id = grad.user_key AND a.nd_key = grad.nd_key
LEFT JOIN enrollments.term_completions AS b ON a.applicant_id = b.user_id AND a.nd_key = b.nd_key
WHERE a.nd_key = 'nd013' AND a.cohort_id in ('47','88','89','18')
  AND a.application_status = 'accept';
"""
sql_2 = """
SELECT distinct a.id,a.applicant_id,a.cohort_id,a.cohort_open_at,a.cohort_close_at,a.cohort_start_at
      ,a.application_created_at,a.submitted_at,a.accepted_at,a.first_charge_created_at
      ,'{tag}' AS tag,regexp_split_to_table(coalesce(lower(q.response),'no response'), ', ') as response
      ,CASE WHEN grad.graduated_at IS NOT NULL THEN 'graduated'
            WHEN b.user_id IS NOT NULL THEN 'term completed'
            WHEN a.first_charge_created_at IS NOT NULL THEN 'paid'
       ELSE 'accepted'
       END AS status
      ,CASE WHEN a.first_charge_created_at IS NOT NULL THEN 'paid' ELSE 'accepted' END AS paid_status
FROM applications a
LEFT JOIN admissions.question_responses q 
  ON a.id = q.application_id AND q.question_id IN ({question})
  AND a.cohort_id = q.cohort_id
LEFT JOIN registrar_v2.graduations grad ON a.applicant_id = grad.user_key AND a.nd_key = grad.nd_key
LEFT JOIN enrollments.term_completions AS b ON a.applicant_id = b.user_id AND a.nd_key = b.nd_key
WHERE a.nd_key = 'nd013' AND a.cohort_id in ('47','88','89','18')
  AND a.application_status = 'accept';
"""

In [4]:
laurel = open('conn_laurel.txt', 'r')
conn_laurel = psycopg2.connect(laurel.read())

_list = ['gender','degree relevant','education','education institution','professional experience','employer','employment','python','C++','programming skills','theory skills','time commitment','goal','purpose','phone','location']

for i in _list:
    print('\n')
    print('Running: ',i)
    question_list = ','.join(np.unique("'"+question[question['tag']==i]['question_id']+"'"))
    if np.unique(question[question['tag']==i]['one-hot'])=='Y':
        sql = sql_2.format(tag=i,question=question_list)
    else:
        sql = sql_1.format(tag=i,question=question_list)
    df = pd.read_sql(sql,conn_laurel)
    
    paid = pd.pivot_table(df,values='id',index=['response'],columns=['paid_status'],aggfunc=np.count_nonzero).reset_index().fillna(0)
    paid['paid/accept'] = paid['paid']/paid['accepted']
    paid['paid_rate'] = paid['paid']/(paid['paid']+paid['accepted'])
    print('\n')
    print('paid in accept')
    print(paid.sort_values(by='paid/accept',ascending=False))
    
    grad = pd.pivot_table(df,values='id',index=['response'],columns=['status'],aggfunc=np.count_nonzero).reset_index().fillna(0)
    grad['term_completed_total'] = grad['term completed']+grad['graduated']
    grad['term_completed_total/paid'] = grad['term_completed_total']/grad['paid']
    grad['term_completion_rate'] = grad['term_completed_total']/(grad['term_completed_total']+grad['paid'])
    print('\n')
    print('term completed in paid')
    print(grad.sort_values(by='term_completed_total/paid',ascending=False))
    



Running:  gender


/anaconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:216: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))




paid in accept
paid_status                 response  accepted  paid  paid/accept  paid_rate
4             prefer not to disclose        73    42     0.575342   0.365217
1                               male      4723  2593     0.549015   0.354429
0                             female       416   209     0.502404   0.334400
2                        no response        71    30     0.422535   0.297030
5            prefer to self-describe        18     6     0.333333   0.250000
3                         non binary        35     5     0.142857   0.125000


term completed in paid
status                 response  accepted  graduated    paid  term completed  \
2                   no response      70.0        0.0    15.0            16.0   
3                    non binary      34.0        0.0     3.0             3.0   
1                          male    4631.0       25.0  1355.0          1305.0   
4        prefer not to disclose      72.0        0.0    23.0            20.0   
0                  



paid in accept
paid_status                           response  accepted    paid  paid/accept  \
2                                     5+ years    1885.0  1378.0     0.731034   
1                                    3-5 years     869.0   512.0     0.589183   
0                                   1-2+ years    1174.0   575.0     0.489779   
3                                     < 1 year     777.0   282.0     0.362934   
4            i have no professional experience     630.0   138.0     0.219048   
5                                  no response       1.0     0.0     0.000000   

paid_status  paid_rate  
2             0.422311  
1             0.370746  
0             0.328759  
3             0.266289  
4             0.179688  
5             0.000000  


term completed in paid
status                           response  accepted  graduated   paid  \
2                                5+ years    1829.0       13.0  712.0   
3                                < 1 year     766.0        3.0  149.0



paid in accept
paid_status                    response  accepted    paid  paid/accept  \
5                        yes, full time    3080.0  2038.0     0.661688   
2                         self employed     257.0   160.0     0.622568   
1                                 other      43.0    25.0     0.581395   
6            yes, part time or contract     345.0   170.0     0.492754   
4                            unemployed     258.0   126.0     0.488372   
3                               student    1352.0   366.0     0.270710   
0                           no response       1.0     0.0     0.000000   

paid_status  paid_rate  
5             0.398202  
2             0.383693  
1             0.367647  
6             0.330097  
4             0.328125  
3             0.213038  
0             0.000000  


term completed in paid
status                    response  accepted  graduated    paid  \
1                            other      43.0        1.0     9.0   
2                    self emplo



paid in accept
paid_status                    response  accepted  paid  paid/accept  \
3                      machine learning      3298  2041     0.618860   
0                              calculus      4357  2641     0.606151   
6            probability and statistics      4476  2687     0.600313   
2                        linear algebra      4544  2687     0.591329   
1                      computer science      4543  2599     0.572089   
4                           no response         4     1     0.250000   
5                     none of the above        74    16     0.216216   

paid_status  paid_rate  
3             0.382281  
0             0.377394  
6             0.375122  
2             0.371595  
1             0.363904  
4             0.200000  
5             0.177778  


term completed in paid
status                    response  accepted  graduated    paid  \
5                none of the above      73.0        0.0     8.0   
3                 machine learning    3213.0   



paid in accept
paid_status                                           response  accepted  \
1                              grow skills for my current role    1419.0   
6                             start a new career in this field    2199.0   
5                                                        other      56.0   
2                          help move from academia to industry     512.0   
0            general interest in the topic (personal growth...     858.0   
3                          help prepare for an advanced degree     291.0   
4                                                  no response       1.0   

paid_status    paid  paid/accept  paid_rate  
1             884.0     0.622974   0.383847  
6            1314.0     0.597544   0.374039  
5              25.0     0.446429   0.308642  
2             228.0     0.445312   0.308108  
0             358.0     0.417249   0.294408  
3              76.0     0.261168   0.207084  
4               0.0     0.000000   0.000000  


term

In [5]:
pd.pivot_table(question, values='question_id', index=['cohort_id'],columns=['tag'], aggfunc=np.unique).T

cohort_id,7,18,47,88,89,90,220,293,359,389
tag,,,,,,,,,,
C,8afe0061-746b-4bd7-807c-393fe5c7599d,None,None,None,None,None,None,None,None,None
C++,None,7a09e78a-8000-459e-9d79-aa21db135138,7a09e78a-8000-459e-9d79-aa21db135138,7a09e78a-8000-459e-9d79-aa21db135138,9481dc6b-c4e9-4207-a30c-65c3ac302bf5,9481dc6b-c4e9-4207-a30c-65c3ac302bf5,9481dc6b-c4e9-4207-a30c-65c3ac302bf5,9481dc6b-c4e9-4207-a30c-65c3ac302bf5,9481dc6b-c4e9-4207-a30c-65c3ac302bf5,9481dc6b-c4e9-4207-a30c-65c3ac302bf5
age,1bfe0061-746b-4bd7-807c-393fe5c7599d,None,None,None,None,None,None,None,None,None
degree relevant,None,cad2565e-8176-44b5-9080-14cfc67ea9b4,cad2565e-8176-44b5-9080-14cfc67ea9b4,cad2565e-8176-44b5-9080-14cfc67ea9b4,cad2565e-8176-44b5-9080-14cfc67ea9b4,cad2565e-8176-44b5-9080-14cfc67ea9b4,cad2565e-8176-44b5-9080-14cfc67ea9b4,cad2565e-8176-44b5-9080-14cfc67ea9b4,cad2565e-8176-44b5-9080-14cfc67ea9b4,cad2565e-8176-44b5-9080-14cfc67ea9b4
education,None,48e7b492-62b4-4d99-b596-80d68f2966ae,48e7b492-62b4-4d99-b596-80d68f2966ae,48e7b492-62b4-4d99-b596-80d68f2966ae,48e7b492-62b4-4d99-b596-80d68f2966ae,48e7b492-62b4-4d99-b596-80d68f2966ae,48e7b492-62b4-4d99-b596-80d68f2966ae,48e7b492-62b4-4d99-b596-80d68f2966ae,48e7b492-62b4-4d99-b596-80d68f2966ae,48e7b492-62b4-4d99-b596-80d68f2966ae
education institution,None,892afb90-16b5-4531-a738-3f4366430224,892afb90-16b5-4531-a738-3f4366430224,892afb90-16b5-4531-a738-3f4366430224,892afb90-16b5-4531-a738-3f4366430224,892afb90-16b5-4531-a738-3f4366430224,892afb90-16b5-4531-a738-3f4366430224,892afb90-16b5-4531-a738-3f4366430224,892afb90-16b5-4531-a738-3f4366430224,892afb90-16b5-4531-a738-3f4366430224
employer,None,aefb0704-b6f1-40ff-a574-bade2df80370,aefb0704-b6f1-40ff-a574-bade2df80370,aefb0704-b6f1-40ff-a574-bade2df80370,aefb0704-b6f1-40ff-a574-bade2df80370,aefb0704-b6f1-40ff-a574-bade2df80370,aefb0704-b6f1-40ff-a574-bade2df80370,aefb0704-b6f1-40ff-a574-bade2df80370,aefb0704-b6f1-40ff-a574-bade2df80370,aefb0704-b6f1-40ff-a574-bade2df80370
employment,None,fba3666b-db04-46e9-8f3d-2a303f13e0a5,fba3666b-db04-46e9-8f3d-2a303f13e0a5,fba3666b-db04-46e9-8f3d-2a303f13e0a5,fba3666b-db04-46e9-8f3d-2a303f13e0a5,fba3666b-db04-46e9-8f3d-2a303f13e0a5,fba3666b-db04-46e9-8f3d-2a303f13e0a5,fba3666b-db04-46e9-8f3d-2a303f13e0a5,fba3666b-db04-46e9-8f3d-2a303f13e0a5,fba3666b-db04-46e9-8f3d-2a303f13e0a5
gender,None,516737fe-4567-48d2-8645-08d335e21b0c,516737fe-4567-48d2-8645-08d335e21b0c,516737fe-4567-48d2-8645-08d335e21b0c,516737fe-4567-48d2-8645-08d335e21b0c,516737fe-4567-48d2-8645-08d335e21b0c,516737fe-4567-48d2-8645-08d335e21b0c,516737fe-4567-48d2-8645-08d335e21b0c,516737fe-4567-48d2-8645-08d335e21b0c,516737fe-4567-48d2-8645-08d335e21b0c


In [6]:
laurel = open('conn_laurel.txt', 'r')
conn_laurel = psycopg2.connect(laurel.read())

### C++

In [7]:
sql_cplus = """
SELECT distinct a.id,a.applicant_id,a.cohort_id,a.cohort_open_at,a.cohort_close_at,a.cohort_start_at
      ,a.application_created_at,a.submitted_at,a.accepted_at,a.first_charge_created_at
      ,'c++' AS tag,q.response
      ,CASE WHEN grad.graduated_at IS NOT NULL THEN 'graduated'
            WHEN b.user_id IS NOT NULL THEN 'term completed'
            WHEN a.first_charge_created_at IS NOT NULL THEN 'paid'
       ELSE 'accepted'
       END AS status
      ,CASE WHEN a.first_charge_created_at IS NOT NULL THEN 'paid' ELSE 'accepte' END AS paid_status
FROM applications a
LEFT JOIN admissions.question_responses q 
  ON a.id = q.application_id AND q.question_id IN ('7a09e78a-8000-459e-9d79-aa21db135138','9481dc6b-c4e9-4207-a30c-65c3ac302bf5')
  AND a.cohort_id = q.cohort_id
LEFT JOIN registrar_v2.graduations grad ON a.applicant_id = grad.user_key AND a.nd_key = grad.nd_key
LEFT JOIN enrollments.term_completions AS b ON a.applicant_id = b.user_id AND a.nd_key = b.nd_key
WHERE a.nd_key = 'nd013' AND a.cohort_id in ('47','88','89','18')
  AND a.application_status = 'accept';
"""
cplus = pd.read_sql(sql_cplus,conn_laurel)

In [8]:
cplus.groupby(['paid_status','status','response'])['id'].count()

paid_status  status          response           
accepte      accepted        1-2 years              1135
                             2+ years               1505
                             < 1 year               1280
                             Level 0                  21
                             Level 1                 104
                             Level 2                 303
                             Level 3                 281
                             Level 4                 157
                             No prior experience     440
             graduated       1-2 years                 1
                             2+ years                  2
                             < 1 year                  2
             term completed  1-2 years                27
                             2+ years                 43
                             < 1 year                 19
                             Level 2                   1
                             Level 3   

In [9]:
pd.pivot_table(cplus, values='id', index=['response'],columns=['paid_status'], aggfunc=np.count_nonzero).reset_index()

paid_status,response,accepte,paid
0,1-2 years,1163,646
1,2+ years,1550,1069
2,< 1 year,1301,635
3,Level 0,21,9
4,Level 1,104,35
5,Level 2,304,106
6,Level 3,286,141
7,Level 4,161,82
8,No prior experience,445,162


In [10]:
pd.pivot_table(cplus, values='id', index=['response'],columns=['status'], aggfunc=np.count_nonzero).reset_index()

status,response,accepted,graduated,paid,term completed
0,1-2 years,1135.0,8.0,350.0,316.0
1,2+ years,1505.0,8.0,527.0,579.0
2,< 1 year,1280.0,8.0,339.0,309.0
3,Level 0,21.0,NaN,7.0,2.0
4,Level 1,104.0,NaN,23.0,12.0
5,Level 2,303.0,NaN,66.0,41.0
6,Level 3,281.0,NaN,78.0,68.0
7,Level 4,157.0,NaN,45.0,41.0
8,No prior experience,440.0,1.0,89.0,77.0


### degree relevant

In [11]:
sql_dr = """
SELECT distinct a.id,a.applicant_id,a.cohort_id,a.cohort_open_at,a.cohort_close_at,a.cohort_start_at
      ,a.application_created_at,a.submitted_at,a.accepted_at,a.first_charge_created_at
      ,'degree relevant' AS tag,q.response,regexp_split_to_table(q.response, ', ') as response_new
      ,CASE WHEN grad.graduated_at IS NOT NULL THEN 'graduated'
            WHEN b.user_id IS NOT NULL THEN 'term completed'
            WHEN a.first_charge_created_at IS NOT NULL THEN 'paid'
       ELSE 'accepted'
       END AS status
      ,CASE WHEN a.first_charge_created_at IS NOT NULL THEN 'paid' ELSE 'accepte' END AS paid_status
FROM applications a
LEFT JOIN admissions.question_responses q 
  ON a.id = q.application_id AND q.question_id IN ('cad2565e-8176-44b5-9080-14cfc67ea9b4')
  AND a.cohort_id = q.cohort_id
LEFT JOIN registrar_v2.graduations grad ON a.applicant_id = grad.user_key AND a.nd_key = grad.nd_key
LEFT JOIN enrollments.term_completions AS b ON a.applicant_id = b.user_id AND a.nd_key = b.nd_key
WHERE a.nd_key = 'nd013' AND a.cohort_id in ('47','88','89','18')
  AND a.application_status = 'accept';
"""
dr = pd.read_sql(sql_dr,conn_laurel)

In [12]:
pd.pivot_table(dr, values='id', index=['response_new'],columns=['paid_status'], aggfunc=np.count_nonzero).reset_index()

paid_status,response_new,accepte,paid
0,Computer science,3018,1556
1,Engineering,3448,1926
2,Mathematics,1020,629
3,Not related/relevant to Nanodegree.,147,73
4,Other,401,268


In [13]:
pd.pivot_table(dr, values='id', index=['response_new'],columns=['status'], aggfunc=np.count_nonzero).reset_index()

status,response_new,accepted,graduated,paid,term completed
0,Computer science,2958.0,16.0,825.0,775.0
1,Engineering,3371.0,15.0,986.0,1002.0
2,Mathematics,1000.0,9.0,330.0,310.0
3,Not related/relevant to Nanodegree.,146.0,NaN,51.0,23.0
4,Other,394.0,3.0,141.0,131.0


### Goal

In [14]:
sql_goal = """
SELECT distinct a.id,a.applicant_id,a.cohort_id,a.cohort_open_at,a.cohort_close_at,a.cohort_start_at
      ,a.application_created_at,a.submitted_at,a.accepted_at,a.first_charge_created_at
      ,'goal' AS tag,coalesce(lower(q.response),'no response') AS response
      ,array_length(regexp_split_to_array(lower(q.response), '\s'),1) As response_len
      ,case when array_length(regexp_split_to_array(lower(q.response), '\s'),1) <= 10 then '0 - 10'
            when array_length(regexp_split_to_array(lower(q.response), '\s'),1) <= 20 then '10 - 20'
            when array_length(regexp_split_to_array(lower(q.response), '\s'),1) <= 30 then '20 - 30'
            when array_length(regexp_split_to_array(lower(q.response), '\s'),1) <= 40 then '30 - 40'
            when array_length(regexp_split_to_array(lower(q.response), '\s'),1) <= 50 then '40 - 50'
            when array_length(regexp_split_to_array(lower(q.response), '\s'),1) <= 80 then '50 - 80'
            when array_length(regexp_split_to_array(lower(q.response), '\s'),1) <= 100 then '80 - 100'
            when array_length(regexp_split_to_array(lower(q.response), '\s'),1) <= 200 then '100 - 200'
            when array_length(regexp_split_to_array(lower(q.response), '\s'),1) <= 300 then '200 - 300'
            when array_length(regexp_split_to_array(lower(q.response), '\s'),1) <= 400 then '300 - 400'
            when array_length(regexp_split_to_array(lower(q.response), '\s'),1) <= 500 then '400 - 500'
            when array_length(regexp_split_to_array(lower(q.response), '\s'),1) > 500 then '> 500'
       else 'others' end AS response_length
      ,CASE WHEN grad.graduated_at IS NOT NULL THEN 'graduated'
            WHEN b.user_id IS NOT NULL THEN 'term completed'
            WHEN a.first_charge_created_at IS NOT NULL THEN 'paid'
       ELSE 'accepted'
       END AS status
      ,CASE WHEN a.first_charge_created_at IS NOT NULL THEN 'paid' ELSE 'accepted' END AS paid_status
FROM applications a
LEFT JOIN admissions.question_responses q 
  ON a.id = q.application_id AND q.question_id IN ('2ad03aaa-1b35-4381-9c43-907b1b4eba67')
  AND a.cohort_id = q.cohort_id
LEFT JOIN registrar_v2.graduations grad ON a.applicant_id = grad.user_key AND a.nd_key = grad.nd_key
LEFT JOIN enrollments.term_completions AS b ON a.applicant_id = b.user_id AND a.nd_key = b.nd_key
WHERE a.nd_key = 'nd013' AND a.cohort_id in ('47','88','89','18')
  AND a.application_status = 'accept'
"""
goal = pd.read_sql(sql_goal,conn_laurel)

In [15]:
paid = pd.pivot_table(goal, values='id', index=['response_length'],columns=['paid_status'], aggfunc=np.count_nonzero).reset_index()
paid['paid/accept'] = paid['paid']/paid['accepted']
paid['paid_rate'] = paid['paid']/(paid['paid']+paid['accepted'])
paid.sort_values(by='paid/accept',ascending=False)

paid_status,response_length,accepted,paid,paid/accept,paid_rate
2,100 - 200,734.0,1003.0,1.366485,0.577432
6,300 - 400,6.0,7.0,1.166667,0.538462
4,200 - 300,51.0,55.0,1.078431,0.518868
10,80 - 100,332.0,302.0,0.909639,0.476341
9,50 - 80,882.0,559.0,0.633787,0.387925
8,400 - 500,2.0,1.0,0.500000,0.333333
7,40 - 50,484.0,204.0,0.421488,0.296512
5,30 - 40,561.0,223.0,0.397504,0.284439
3,20 - 30,666.0,225.0,0.337838,0.252525
1,10 - 20,848.0,193.0,0.227594,0.185399


In [16]:
grad = pd.pivot_table(goal,values='id',index=['response_length'],columns=['status'],aggfunc=np.count_nonzero).reset_index().fillna(0)
grad['term_completed_total'] = grad['term completed']+grad['graduated']
grad['term_completed_total/paid'] = grad['term_completed_total']/grad['paid']
grad['term_completion_rate'] = grad['term_completed_total']/(grad['term_completed_total']+grad['paid'])
grad.sort_values(by='term_completed_total/paid',ascending=False)

status,response_length,accepted,graduated,paid,term completed,term_completed_total,term_completed_total/paid,term_completion_rate
8,400 - 500,2.0,0.0,0.0,1.0,1.0,inf,1.000000
4,200 - 300,45.0,0.0,25.0,36.0,36.0,1.440000,0.590164
0,0 - 10,749.0,3.0,58.0,62.0,65.0,1.120690,0.528455
2,100 - 200,716.0,6.0,484.0,531.0,537.0,1.109504,0.525955
10,80 - 100,322.0,3.0,149.0,160.0,163.0,1.093960,0.522436
6,300 - 400,5.0,0.0,4.0,4.0,4.0,1.000000,0.500000
5,30 - 40,544.0,4.0,126.0,110.0,114.0,0.904762,0.475000
1,10 - 20,836.0,4.0,109.0,92.0,96.0,0.880734,0.468293
9,50 - 80,864.0,1.0,311.0,265.0,266.0,0.855305,0.461005
7,40 - 50,475.0,1.0,123.0,89.0,90.0,0.731707,0.422535
